# Using the default resnet

## Imports

In [1]:
import pandas as pd
import torch
import os
import matplotlib.pyplot as plt
from torch import Tensor
from scipy import ndimage
from PIL import Image
import numpy as np
#from typing import Type, Any, Callable, Union, List, Optional, Tuple
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset,Subset
from torchvision.models import resnet18

c:\ProgramData\miniconda3\envs\mphys-galaxy\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Parameters

In [2]:
USE_GPU = True
CATALOG_PATH = '../Data/subset_gz1_desi_cross_cat.csv'
DATA_PATH = '../Data/Subset'

In [3]:
#Run processes on CPU or GPU
if USE_GPU:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device.type == 'cuda':
        print(torch.cuda.get_device_name(0))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
else:
    device = torch.device('cpu')

print('Using device:', device)

Using device: cpu


## Importing data

In [4]:
catalog = pd.read_csv(CATALOG_PATH)
print(f"Loaded {catalog.shape[0]} galaxy images")

#Select features (clockwise and anti-clockwise probabilities)
Y = catalog[['P_CW','P_ACW']]
classes = [r'P_CW',r'P_ACW']

Loaded 1500 galaxy images


### Generate list of file locations from catalog

In [5]:
def get_file_paths(catalog_to_convert,folder_path ):
    brick_ids = catalog_to_convert['dr8_id'].str.split("_",expand=True)[0]
    dr8_ids = catalog_to_convert['dr8_id']
    file_locations = folder_path+'/'+brick_ids+'/'+dr8_ids+'.jpg'
    return file_locations

file_locations = get_file_paths(catalog,DATA_PATH)

### Image processing functions

In [6]:
COLOR = True
RAW_SIZE = 15
IMG_SIZE = 160

TARGET_SIZE = 5
TRANSLATE = 0.
ROTATE = False
FLIP = False
LABEL = 2
SHUFFLE = False


def img_proc(img, raw_size=RAW_SIZE, target_size=TARGET_SIZE, translate=TRANSLATE, rotate=ROTATE,
             random_generator=None):
    assert img.shape[-2] == img.shape[-1]
    assert translate >= 0
    
    if random_generator is None:
        random_generator = int.from_bytes(os.urandom(4), byteorder='little')
    rng = np.random.default_rng(random_generator)

    if not isinstance(target_size, (float, int)):
        target_size = rng.uniform(*target_size)
    if rotate:
        assert target_size * (1 + translate) * 2**0.5 < raw_size
        if isinstance(rotate, bool):
            rotate = rng.uniform(0., 360.)
        img = ndimage.rotate(img, rotate, axes=(-1, -2), reshape=False, order=1)
    else:
        assert target_size * (1 + translate) < raw_size
    s = img.shape[-1]
    translate = translate * target_size
    t_x = rng.uniform(-translate, translate) if translate > 0 else 0
    t_y = rng.uniform(-translate, translate) if translate > 0 else 0
    a = int(s * (raw_size - target_size + t_x) / (2 * raw_size))
    b = int(s * (raw_size - target_size + t_y) / (2 * raw_size))
    c = int(s * target_size / raw_size)
    return img[..., a:(a + c), b:(b + c)]


def read_img(path, color=COLOR, img_size=IMG_SIZE, atleast_3d=True, random_flip=FLIP,
             shuffle_channel=SHUFFLE, random_generator=None, **kwargs):
    if random_generator is None:
        random_generator = int.from_bytes(os.urandom(4), byteorder='little')
    rng = np.random.default_rng(random_generator)

    jpeg_file = (np.asarray(Image.open(path)) / 255).astype(np.float32)
    if color:
        img = np.moveaxis(jpeg_file, -1, 0)
        if shuffle_channel:
            if isinstance(shuffle_channel, bool):
                shuffle_channel = rng.permutation(img.shape[0])
            img = img[shuffle_channel]
    else:
        img = np.mean(jpeg_file, axis=-1)
    img = img_proc(img, random_generator=rng, **kwargs)
    rng.uniform(size=100) # just to jump the rng

    if img_size is not None:
        z = img_size / img.shape[-1]
        if img.ndim == 2:
            img = ndimage.zoom(img, (z, z), order=1)
            if atleast_3d:
                img = img[np.newaxis]
        elif img.ndim == 3:
            img = ndimage.zoom(img, (1, z, z), order=1)
        else:
            raise RuntimeError

    if random_flip:
        flip = int(rng.integers(0, 2, 1))
        flip = 2 * flip - 1
    else:
        flip = 1
    return np.ascontiguousarray(img[..., ::flip])

### Reading in images

In [7]:
#Takes 2 mins 20
X = torch.empty(0, 3, 160,160)
for i in range(len(file_locations)):
    X = torch.cat((X, torch.from_numpy(read_img(file_locations[i])).float()[np.newaxis]), 0)

In [8]:
print(f"Processed {X.shape[0]} images")

Processed 1500 images


## Setting up model

In [9]:
selected_model = resnet18(num_classes=2)
if torch.cuda.is_available():
    selected_model.cuda()

### Hyperparameters

In [10]:
learning_rate = 0.0001
weight_decay = 1

num_epochs = 120
batch_size = 60 #60

modfile = 'resnet18_modfile.pt'

### Training model function

In [11]:
def train(model, trainloader, optimiser, device):
    train_loss = 0.0
    model.train()

    for batch_idx, (data, labels) in enumerate(trainloader):
        data, labels = data.to(device), labels.to(device)
        optimiser.zero_grad()

        #Calculate train loss
        #Softmax output and labels for passing to cross-entropy function
        p_y = model(data).softmax(dim=1)
        labels = labels.softmax(dim=1).to(device) 

        loss_criterion = nn.CrossEntropyLoss()
        loss = loss_criterion(p_y, labels)
            
        train_loss += loss.item() * data.size(0)
        
        #Feed the loss back
        loss.backward()
        optimiser.step()

    train_loss /= len(trainloader.dataset)
    return train_loss

### Validating model function

In [12]:
#function to test the model on data
def validate(model, testloader, device):
    correct = 0
    test_loss = 0.0
    mse_loss = 0.0

    model.eval()
    with torch.no_grad():
        for batch_idx, (data, labels) in enumerate(testloader):
            data, labels = data.to(device), labels.to(device)

            #Softmax output and labels for passing to cross-entropy function
            p_y = model(data).softmax(dim=1)
            labels = labels.to(device)

            #Calculate test loss         
            loss_criterion = nn.CrossEntropyLoss()
            loss = loss_criterion(p_y, labels)
            test_loss += loss.item() * data.size(0)

        test_loss /= len(testloader.dataset)
        
    return test_loss

### Split data into train and test

In [13]:
def data_split(frac_val, data):
    dataset_size = len(data)
    nval = int(frac_val*dataset_size)

    indices = list(range(dataset_size))
    train_indices, val_indices = indices[nval:], indices[:nval]

    train_sampler = Subset(data, train_indices)
    valid_sampler = Subset(data, val_indices)

    train_loader = torch.utils.data.DataLoader(train_sampler, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(valid_sampler, batch_size=batch_size, shuffle=True)
    return train_loader, test_loader

## Running model

In [14]:
#function to run the entire model train/test loop
TEST_USE = False
early_stopping = True
QUIET = False

def test_train_model(X,Y,model):
    X_tensor = X
    Y_tensor = torch.from_numpy(Y.values).float()

    dataset = TensorDataset(X_tensor, Y_tensor)
    train_loader,val_loader = data_split(0.7,dataset)

    if TEST_USE: #Doesn't currently work
        val_loader, test_loader = data_split(0.5,val_loader)

    #optimizer and learning rate scheduler
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay = weight_decay)#0.005, #0.01
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.85)

    #array to store metrics
    result_arr = np.zeros((num_epochs,4))

    _bestloss = 1.
    #NN learning
    for epoch in range(num_epochs):
    
        train_loss = train(model, train_loader, optimizer, device)
        val_loss = validate(model, val_loader, device)
        
        #scheduler.step() 

        if early_stopping and val_loss<_bestloss:
            _bestloss = val_loss
            torch.save(model.state_dict(), modfile)
            best_epoch = epoch

        #set output row
        results = [epoch, train_loss, val_loss]
        result_arr[epoch] = results

        #print epoch results
        if not QUIET:
            print(f"Epoch: {epoch}, Train Loss: {train_loss:4f}, Validation Loss: {val_loss:4f}, Learning Rate: {optimizer.param_groups[0]['lr']:4f}")
    
    test_arr=[]
    if TEST_USE:
        #Test data run
        bestmodel=selected_model().to(device)
        bestmodel.load_state_dict(torch.load('modfile.pt'))
        test_loss, test_accuracy, test_auc, prediction2,target2,  recall2, f12, precision2 = validate(bestmodel, test_loader, device)
        test_arr=[test_loss, test_accuracy, test_auc, prediction2,target2,  recall2, f12, precision2]

    if not early_stopping:
        torch.save(model.state_dict(), modfile)
        best_epoch = -1

    return result_arr, test_arr, best_epoch

## Actually run model

In [15]:
result_arr1, test_arr1, best_epoch = test_train_model(X,Y,selected_model)

c:\ProgramData\miniconda3\envs\mphys-galaxy\Lib\site-packages\torch\optim\lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch: 0, Train Loss: 0.020931, Validation Loss: 0.618887, MSE Loss: 0.378476
Epoch: 1, Train Loss: 0.013708, Validation Loss: 0.617042, MSE Loss: 0.353710
Epoch: 2, Train Loss: 0.012033, Validation Loss: 0.660202, MSE Loss: 0.420842
Epoch: 3, Train Loss: 0.009693, Validation Loss: 0.627378, MSE Loss: 0.341717
Epoch: 4, Train Loss: 0.007285, Validation Loss: 0.648701, MSE Loss: 0.436407
Epoch: 5, Train Loss: 0.005792, Validation Loss: 0.667259, MSE Loss: 0.409309
Epoch: 6, Train Loss: 0.005397, Validation Loss: 0.735329, MSE Loss: 0.667150
Epoch: 7, Train Loss: 0.006796, Validation Loss: 1.060741, MSE Loss: 1.531729
Epoch: 8, Train Loss: 0.007640, Validation Loss: 0.761429, MSE Loss: 0.933255
Epoch: 9, Train Loss: 0.007148, Validation Loss: 0.815346, MSE Loss: 0.847575
Epoch: 10, Train Loss: 0.006320, Validation Loss: 0.744167, MSE Loss: 0.392949
Epoch: 11, Train Loss: 0.006130, Validation Loss: 0.881675, MSE Loss: 1.012219
Epoch: 12, Train Loss: 0.006646, Validation Loss: 0.877181, MS

KeyboardInterrupt: 

In [16]:
#plot the training cycle
plt.figure(figsize=(10,5))
plt.plot(result_arr1[:,0],result_arr1[:,3],linestyle='-', label='Accuracy')
plt.plot(result_arr1[:,0],result_arr1[:,2],linestyle='-', c='orange',label='Validation loss')
plt.plot(result_arr1[:,0],result_arr1[:,1],linestyle='-', c='violet',label='Train loss')
plt.ylim(0,1.5)
plt.xlim(0,num_epochs-1)
plt.title('NN training cycle')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.legend()
plt.show()

NameError: name 'result_arr1' is not defined

<Figure size 1000x500 with 0 Axes>